In [ ]:
import pandas as pd
#from mlxtend.preprocessing import OnehotTransactions
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [47]:
df = pd.read_csv('../data/import ine 2020.csv')
df.head()

,gestion,mes,mes.1,partida,descripcion
0,2020,1,Enero,402,leche
1,2020,9,Septiembre,402,leche
2,2020,1,Enero,401,leche
3,2020,1,Enero,402,leche
4,2020,1,Enero,402,leche


In [ ]:
dataset = [['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']]
